#  SQL Case Study

In [1]:
import os

In [2]:
os.getcwd()

'/Users/andreafung/Dropbox/Springboard/sql case study'

Using SQLite3 to import data

In [1]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """SELECT *
    FROM Facilities


        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "/Users/andreafung/Dropbox/Springboard/sql case study/sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


## Another way to import SQL data by using SQLAlchemy

Using sqlaclchemy and pandas to import data

In [2]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd


In [3]:
# create engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [4]:
#creating a definition to use in running sqlite coding.  
def run_query(engine, q):
    with engine.connect() as con:
        rs = con.execute(q)
        df = pd.DataFrame(rs.fetchmany(size=5))
        df.columns = rs.keys()
    return df

### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [5]:

q = """ 
    SELECT 
     F.name
    ,SUM(CASE WHEN M.memid = 0 THEN F.guestcost ELSE F.membercost END * B.slots ) AS revenue
    FROM Bookings B
    INNER JOIN Facilities F ON B.facid = F.facid
    LEFT JOIN Members M ON B.memid = M.memid
    GROUP BY 
    F.name
    HAVING(revenue) < 1000
    ORDER BY revenue
    """ 
run_query(engine, q)

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 



In [6]:
#playing with the sql
q="""SELECT surname, firstname, recommendedby
    FROM Members
    WHERE memid>'0'
    ORDER BY surname, firstname"""
run_query(engine,q)

,surname,firstname,recommendedby
0,Bader,Florence,9
1,Baker,Anne,9
2,Baker,Timothy,13
3,Boothe,Tim,3
4,Butters,Gerald,1


### Q12: Find the facilities with their usage by member, but not guests

In [15]:
q="""SELECT (M.firstname||' '|| M.surname) AS Fullname, name AS Facility, SUM(B.slots) as Usage
 FROM Members AS M
JOIN Bookings AS B ON M.memid = B.memid
JOIN Facilities ON B.facid = Facilities.facid
WHERE B.memid>'0'
GROUP BY name 
ORDER BY Fullname
"""

run_query(engine,q)

,Fullname,Facility,Usage
0,Darren Smith,Badminton Court,1086
1,Darren Smith,Massage Room 1,884
2,Darren Smith,Pool Table,856
3,Darren Smith,Snooker Table,860
4,Darren Smith,Squash Court,418


###  Q13: Find the facilities usage by month, but not guests

In [12]:
q=""" SELECT F.name AS Facility, SUM(B.slots) as Usage, 
strftime('%Y-%m', B.starttime) as month

 FROM Bookings AS B
INNER JOIN Facilities AS F ON B.facid = F.facid
INNER JOIN Members as M ON B.memid=M.memid
WHERE M.memid>'0'
GROUP BY Facility, month
ORDER BY usage DESC
"""

run_query(engine,q)

,Facility,Usage,month
0,Badminton Court,507,2012-09
1,Pool Table,443,2012-09
2,Tennis Court 1,417,2012-09
3,Badminton Court,414,2012-08
4,Tennis Court 2,414,2012-09
